In [ ]:
import numpy as np
from PIL import Image
import time
import functools

%tensorflow_version 1.x
import tensorflow as tf

from tensorflow.python.keras.preprocessing import image as kp_image
from tensorflow.python.keras import models as Kmodels 
from tensorflow.python.keras import losses as Klosses
from tensorflow.python.keras import layers as Klayers
from tensorflow.python.keras import backend as K

# Content layer where will pull our feature maps
content_layers = ['block5_conv2'] 

# Style layer we are interested in
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1'
               ]

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

def get_model():
    """ Creates our model with access to intermediate layers. 
  
    This function will load the VGG19 model and access the intermediate layers. 
    These layers will then be used to create a new model that will take input image
        and return the outputs from these intermediate layers from the VGG model. 
  
    Returns:
        returns a keras model that takes image inputs and outputs the style and 
            content intermediate layers. 
    """
    # Load our model. We load pretrained VGG, trained on imagenet data
    vgg = tf.keras.applications.vgg19.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False
    # Get output layers corresponding to style and content layers 
    
    #??? Not really understand "name"...
    style_outputs = [vgg.get_layer(name).output for name in style_layers]
    content_outputs = [vgg.get_layer(name).output for name in content_layers]
    model_outputs = style_outputs + content_outputs
    # Build model 
    return Kmodels.Model(vgg.input, model_outputs)

def get_content_loss(base_content, target):
    return tf.reduce_mean(tf.square(base_content - target))

def gram_matrix(input_tensor):
    # We make the image channels first 
    channels = int(input_tensor.shape[-1])
    a = tf.reshape(input_tensor, [-1, channels])
    n = tf.shape(a)[0]
    gram = tf.matmul(a, a, transpose_a=True)
    return gram / tf.cast(n, tf.float32)


# I changed here!!!!!!!!!!!!!

def get_style_loss(base_style, gram_target, mask_base, mask_target):
    # Expects three images of dimension h, w, c
    # height, width, num filters of each layer
    # We scale the loss ***at a given layer*** by the size of the feature map and the number of filters
    # Do element-wise matrix multiplication for each layer
    
    height, width, channels = base_style.get_shape().as_list()
    base_style_masked = base_style * mask_base
    gram_target_masked = gram_target * mask_target
    gram_style_masked = gram_matrix(base_style_masked)
  
    return tf.reduce_mean(tf.square(gram_style_masked - gram_target_masked))# / (4. * (channels ** 2) * (width * height) ** 2)


def compute_loss(model, loss_weights, init_image, gram_style_features, content_features, mask_base, mask_target):
    """This function will compute the loss total loss.
  
    Arguments:
    model: The model that will give us access to the intermediate layers
    loss_weights: The weights of each contribution of each loss function. 
      (style weight, content weight, and total variation weight)
    init_image: Our initial base image. This image is what we are updating with 
      our optimization process. We apply the gradients wrt the loss we are 
      calculating to this image.
    gram_style_features: Precomputed gram matrices corresponding to the 
      defined style layers of interest.
    content_features: Precomputed outputs from defined content layers of 
      interest.
      
    Returns:
    returns the total loss, style loss, content loss, and total variational loss
    """
    style_weight, content_weight = loss_weights
  
    # Feed our init image through our model. This will give us the content and 
    # style representations at our desired layers. Since we're using eager
    # our model is callable just like any other function!
    model_outputs = model(init_image)
  
    style_output_features = model_outputs[:num_style_layers]
    content_output_features = model_outputs[num_style_layers:]
  
    style_score = 0
    content_score = 0

    # Accumulate style losses from all layers
    # Here, we equally weight each contribution of each loss layer
    weight_per_style_layer = 1.0 / float(num_style_layers)
    for target_style, comb_style in zip(gram_style_features, style_output_features):
        
        # I also changed here! for consistency...
        
        style_score += weight_per_style_layer * get_style_loss(comb_style[0], target_style, mask_base, mask_target)
    
    # Accumulate content losses from all layers 
    weight_per_content_layer = 1.0 / float(num_content_layers)
    for target_content, comb_content in zip(content_features, content_output_features):
        content_score += weight_per_content_layer* get_content_loss(comb_content[0], target_content)
  
    style_score *= style_weight
    content_score *= content_weight

    # Get total loss
    loss = style_score + content_score
    return loss


def style_transfer_loss(content_im, 
                       style_im,
                       mask_base,
                       mask_target):
    
    content_weight=1e3 
    style_weight=1e-2
    
    model = get_model() 
    for layer in model.layers:
        layer.trainable = False
        
    style_output = model(style_im)
    content_output = model(content_im)
    
    style_features = [style_layer[0] for style_layer in style_outputs[:num_style_layers]]
    content_features = [content_layer[0] for content_layer in content_outputs[num_style_layers:]]

    gram_style_features = [gram_matrix(style_feature) for style_feature in style_features]
    
    init_image = content_im
    
    # Create a nice config 
    loss_weights = (style_weight, content_weight)
    cfg = {
      'model': model,
      'loss_weights': loss_weights,
      'init_image': init_image,
      'gram_style_features': gram_style_features,
      'content_features': content_features,
      'mask_base': mask_base, 
      'mask_target': mask_target
    }

    return compute_loss(**cfg)
    